# Deep Learning for Movie Recommendation
In this notebook, I'll build a deep learning model movie recommendations system on the MovieLens 1M dataset.

This will be my 3rd attempt doing this. 
* In the [1st attempt](https://github.com/khanhnamle1994/movielens/blob/master/Content_Based_and_Collaborative_Filtering_Models.ipynb), I tried out content-based and memory-based collaboratice filtering, which rely on the calculation of users and movies' similarity scores. As no training or optimization is involved, these are easy to use approaches. But their performance decrease when we have sparse data which hinders scalability of these approaches for most of the real-world problems.
* In the [2nd attempt](https://github.com/khanhnamle1994/movielens/blob/master/SVD_Model.ipynb), I tried out a matrix factorization model-based collaborative filtering approach called Singular Vector Decomposition, which reduces the dimension of the dataset and gives low-rank approximation of user tastes and preferences.

In this post, I will use a Deep Learning / Neural Network approach that is up and coming with recent development in machine learning and AI technologies.

![movixai](images/movixai.png)

## Loading Datasets
Similar to what I did for the previous notebooks, I loaded the 3 datasets into 3 dataframes: *ratings*, *users*, and *movies*. Additionally, to make it easy to use series from the *ratings* dataframe as training inputs and output to the Keras model, I set *max_userid* as the max value of user_id in the ratings and *max_movieid* as the max value of movie_id in the ratings.

In [4]:
# Import libraries
%matplotlib inline
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Embedding, Reshape, dot, Input
from tensorflow.keras.models import Sequential, Model

# Reading ratings file
ratings = pd.read_csv('ratings.csv', sep='\t', encoding='latin-1', 
                      usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])
max_userid = ratings['user_id'].drop_duplicates().max()
max_movieid = ratings['movie_id'].drop_duplicates().max()

# Reading ratings file
users = pd.read_csv('users.csv', sep='\t', encoding='latin-1', 
                    usecols=['user_id', 'gender', 'zipcode', 'age_desc', 'occ_desc'])

# Reading ratings file
movies = pd.read_csv('movies.csv', sep='\t', encoding='latin-1', 
                     usecols=['movie_id', 'title', 'genres'])

## Matrix Factorization for Collaborative Filtering
I have discussed this extensively in my 2nd notebook, but want to revisit it here as the baseline approach. The idea behind matrix factorization models is that attitudes or preferences of a user can be determined by a small number of hidden factors. We can call these factors as **Embeddings**.

Intuitively, we can understand embeddings as low dimensional hidden factors for movies and users. For e.g. say we have 3 dimensional embeddings for both movies and users.

For instance, for movie A, the 3 numbers in the movie embedding matrix represent 3 different characteristics about the movie, such as:
* How recent is the movie A?
* How much special effects are in movie A?
* How CGI-driven is movie A? 

For user B, the 3 numbers in the user embedding matrix represent:
* How much does user B like Drama movie?
* How likely does user B to give a 5-star rating?
* How often does user B watch movies?

![matrix-factorization](images/matrix-factorization.png)

I definitely would want to create a training and validation set and optimize the number of embeddings by minimizing the RMSE. Intuitively, the RMSE will decrease on the training set as the number of embeddings increases (because I'm approximating the original ratings matrix with a higher rank matrix). Here I create a training set by shuffling randomly the values from the original ratings dataset.

In [2]:
# Create training set
shuffled_ratings = ratings.sample(frac=1., random_state=np.random.randint(1, 100))

# Shuffling users
Users = shuffled_ratings['user_emb_id'].values
print 'Users:', Users, ', shape =', Users.shape

# Shuffling movies
Movies = shuffled_ratings['movie_emb_id'].values
print 'Movies:', Movies, ', shape =', Movies.shape

# Shuffling ratings
Ratings = shuffled_ratings['rating'].values
print 'Ratings:', Ratings, ', shape =', Ratings.shape

Users: [2275 1469  827 ... 5093 3355  194] , shape = (1000209,)
Movies: [2290 3943 3860 ... 1960 1249 2676] , shape = (1000209,)
Ratings: [2 1 3 ... 5 5 5] , shape = (1000209,)


## Deep Learning Model
The idea of using deep learning is similar to that of Model-Based Matrix Factorization. In matrix factorizaion, we decompose our original sparse matrix into product of 2 low rank orthogonal matrices. For deep learning implementation, we don’t need them to be orthogonal, we want our model to learn the values of embedding matrix itself. The user latent features and movie latent features are looked up from the embedding matrices for specific movie-user combination. These are the input values for further linear and non-linear layers. We can pass this input to multiple relu, linear or sigmoid layers and learn the corresponding weights by any optimization algorithm (Adam, SGD, etc.).

### Build the Model
I created a sparse matrix factoring algorithm in Keras for my model in [CFModel.py](https://github.com/khanhnamle1994/movielens/blob/master/CFModel.py). Here are the main components:

* A left embedding layer that creates a Users by Latent Factors matrix.
* A right embedding layer that creates a Movies by Latent Factors matrix.
* When the input to these layers are (i) a user id and (ii) a movie id, they'll return the latent factor vectors for the user and the movie, respectively.
* A merge layer that takes the dot product of these two latent vectors to return the predicted rating.

This code is based on the approach outlined in [Alkahest](http://www.fenris.org/)'s blog post [Collaborative Filtering in Keras](http://www.fenris.org/2016/03/07/index-html).

![embedding-layers](images/embedding-layers.png)

In [5]:
# Import Keras libraries
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint

In [6]:
# Define constants
K_FACTORS = 100 # The number of dimensional embeddings for movies and users
TEST_USER = 2000 # A random test user (user_id = 2000)

I then compile the model using Mean Squared Error (MSE) as the loss function and the AdaMax learning algorithm.

In [7]:
user_input = Input(shape=(1,))
user_embedding = Embedding(max_userid, K_FACTORS, input_length=1)(user_input)
user_embedding = Reshape((K_FACTORS,))(user_embedding)

# Q is the embedding layer that creates a Movie by latent factors matrix.
# If the input is a movie_id, Q returns the latent factor vector for that movie.
movie_input = Input(shape=(1,))
movie_embedding = Embedding(max_movieid, K_FACTORS, input_length=1)(movie_input)
movie_embedding = Reshape((K_FACTORS,))(movie_embedding)

dot_product = dot([user_embedding, movie_embedding], axes=1)

model = Model(inputs=[user_input, movie_input], outputs=dot_product)
model.compile(loss='mse', optimizer='adamax')

# The rate function to predict user's rating of unrated items
def rate(self, user_id, item_id):
    return self.predict([np.array([user_id]), np.array([item_id])])[0][0]

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### Train the Model
Now I need to train the model. This step will be the most-time consuming one. In my particular case, for our dataset with nearly 1 million ratings, almost 6,000 users and 4,000 movies, I trained the model in roughly 6 minutes per epoch (30 epochs ~ 3 hours) inside my Macbook Laptop CPU. I spitted the training and validataion data with ratio of 90/10.

In [8]:
# Callbacks monitor the validation loss
# Save the model weights each time the validation loss has improved
callbacks = [EarlyStopping('val_loss', patience=2), 
             ModelCheckpoint('weights.h5', save_best_only=True)]

# Use 30 epochs, 90% training data, 10% validation data 
history = model.fit([Users, Movies], Ratings, nb_epoch=30, validation_split=.1, verbose=2, callbacks=callbacks)

Train on 900188 samples, validate on 100021 samples
Epoch 1/30
900188/900188 - 147s - loss: 13.8038 - val_loss: 12.5460
Epoch 2/30
900188/900188 - 146s - loss: 8.4083 - val_loss: 5.2729
Epoch 3/30
900188/900188 - 140s - loss: 3.9072 - val_loss: 2.9664
Epoch 4/30
900188/900188 - 134s - loss: 2.4491 - val_loss: 2.0759
Epoch 5/30
900188/900188 - 127s - loss: 1.8172 - val_loss: 1.6404
Epoch 6/30
900188/900188 - 131s - loss: 1.4854 - val_loss: 1.3950
Epoch 7/30
900188/900188 - 129s - loss: 1.2890 - val_loss: 1.2422
Epoch 8/30
900188/900188 - 130s - loss: 1.1627 - val_loss: 1.1408
Epoch 9/30
900188/900188 - 132s - loss: 1.0766 - val_loss: 1.0693
Epoch 10/30
900188/900188 - 127s - loss: 1.0148 - val_loss: 1.0173
Epoch 11/30
900188/900188 - 127s - loss: 0.9686 - val_loss: 0.9776
Epoch 12/30
900188/900188 - 126s - loss: 0.9325 - val_loss: 0.9465
Epoch 13/30
900188/900188 - 128s - loss: 0.9037 - val_loss: 0.9210
Epoch 14/30
900188/900188 - 125s - loss: 0.8797 - val_loss: 0.9004
Epoch 15/30
90018

### Root Mean Square Error
During the training process above, I saved the model weights each time the validation loss has improved. Thus, I can use that value to calculate the best validation Root Mean Square Error.

In [9]:
# Show the best validation RMSE
min_val_loss, idx = min((val, idx) for (idx, val) in enumerate(history.history['val_loss']))
print 'Minimum RMSE at epoch', '{:d}'.format(idx+1), '=', '{:.4f}'.format(math.sqrt(min_val_loss))

Minimum RMSE at epoch 30 = 0.8776


The best validation loss is *0.7424* at epoch 17. Taking the square root of that number, I got the RMSE value of *0.8616*, which is better than the RMSE from the SVD Model (*0.8736*).

### Predict the Ratings
The next step is to actually predict the ratings a random user will give to a random movie. Below I apply the freshly trained deep learning model for all the users and all the movies, using 100 dimensional embeddings for each of them. I also load pre-trained weights from *[weights.h5](https://github.com/khanhnamle1994/movielens/blob/master/weights.h5)* for the model.

In [11]:
# Use the pre-trained model
trained_model = Model(inputs=[user_input, movie_input], outputs=dot_product)
# Load weights
trained_model.load_weights('weights.h5')

As mentioned above, my random test user is has ID 2000.

In [12]:
# Pick a random test user
users[users['user_id'] == TEST_USER]

,user_id,gender,zipcode,age_desc,occ_desc
1999,2000,M,44685,18-24,college/grad student


Here I define the function to predict user's rating of unrated items, using the *rate* function inside the CFModel class in *[CFModel.py](https://github.com/khanhnamle1994/movielens/blob/master/CFModel.py)*.

In [17]:
# Function to predict the ratings given User ID and Movie ID
def predict_rating(user_id, movie_id):
    return rate(trained_model, user_id - 1, movie_id - 1)

Here I show the top 20 movies that user 2000 has already rated, including the *predictions* column showing the values that used 2000 would have rated based on the newly defined *predict_rating* function.

In [18]:
user_ratings = ratings[ratings['user_id'] == TEST_USER][['user_id', 'movie_id', 'rating']]
user_ratings['prediction'] = user_ratings.apply(lambda x: predict_rating(TEST_USER, x['movie_id']), axis=1)
user_ratings.sort_values(by='rating', 
                         ascending=False).merge(movies, 
                                                on='movie_id', 
                                                how='inner', 
                                                suffixes=['_u', '_m']).head(20)

,user_id,movie_id,rating,prediction,title,genres
0,2000,1639,5,3.891420,Chasing Amy (1997),Drama|Romance
1,2000,2529,5,3.829311,Planet of the Apes (1968),Action|Sci-Fi
2,2000,1136,5,4.467747,Monty Python and the Holy Grail (1974),Comedy
3,2000,2321,5,3.830103,Pleasantville (1998),Comedy
4,2000,2858,5,4.455487,American Beauty (1999),Comedy|Drama
5,2000,2501,5,4.240214,October Sky (1999),Drama
6,2000,2804,5,4.271887,"Christmas Story, A (1983)",Comedy|Drama
7,2000,1688,5,3.704615,Anastasia (1997),Animation|Children's|Musical
8,2000,1653,5,3.910572,Gattaca (1997),Drama|Sci-Fi|Thriller
9,2000,527,5,4.729064,Schindler's List (1993),Drama|War


No surpise that these top movies all have 5-start rating. Some of the prediction values seem off (those with value 3.7, 3.8, 3.9 etc.).

### Recommend Movies
Here I make a recommendation list of unrated 20 movies sorted by prediction value for user 2000. Let's see it.

In [19]:
recommendations = ratings[ratings['movie_id'].isin(user_ratings['movie_id']) == False][['movie_id']].drop_duplicates()
recommendations['prediction'] = recommendations.apply(lambda x: predict_rating(TEST_USER, x['movie_id']), axis=1)
recommendations.sort_values(by='prediction',
                          ascending=False).merge(movies,
                                                 on='movie_id',
                                                 how='inner',
                                                 suffixes=['_u', '_m']).head(20)

,movie_id,prediction,title,genres
0,2905,4.751773,Sanjuro (1962),Action|Adventure
1,2019,4.679988,Seven Samurai (The Magnificent Seven) (Shichin...,Action|Drama
2,858,4.653060,"Godfather, The (1972)",Action|Crime|Drama
3,1204,4.630428,Lawrence of Arabia (1962),Adventure|War
4,1178,4.622255,Paths of Glory (1957),Drama|War
5,745,4.615073,"Close Shave, A (1995)",Animation|Comedy|Thriller
6,3022,4.610095,"General, The (1927)",Comedy
7,1250,4.608337,"Bridge on the River Kwai, The (1957)",Drama|War
8,1148,4.578419,"Wrong Trousers, The (1993)",Animation|Comedy
9,750,4.575931,Dr. Strangelove or: How I Learned to Stop Worr...,Sci-Fi|War


## Conclusion
In this notebook, I showed how to use a simple deep learning approach to build a recommendation engine for the MovieLens 1M dataset. This model performed better than all the approaches I attempted before (content-based, user-item similarity collaborative filtering, SVD). I can certainly improve this model's performance by making it deeper with more linear and non-linear layers. I leave that task to you then!